In [ ]:
# Install the package directly from GitHub
!pip install git+https://github.com/wcw100168/Cubed-Sphere-DG-Solver.git

# Getting Started: Scalar Advection

This tutorial demonstrates the basic usage of the **Cubed-Sphere Advection Solver**.
We will simulate the transport of a passive scalar (a Gaussian hill) around the sphere using the Discontinuous Galerkin method.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from cubed_sphere.solvers import CubedSphereAdvectionSolver, AdvectionConfig

> **Note: Dynamic Time-Stepping**
> Notice that we do not specify a `dt` (time step) in the configuration below. The solver features an auto-compute engine that evaluates the initial physical state and the DG Courant-Friedrichs-Lewy (CFL) restriction to automatically determine the safest and most efficient time step.

In [ ]:
# 1. Configure the Solver with Stability Constraints
# For DG on Cubed Sphere, time step scales with 1/N^2
N = 16
R = 1.0
u_max = 2 * np.pi # Approx max velocity for solid body rotation
target_cfl = 0.5

# Note: We no longer manually calculate dt here.
# The solver will compute it dynamically based on the state.

config = AdvectionConfig(
    N=N, 
    CFL=target_cfl, 
    backend='numpy'
    # dt is auto-computed
)

solver = CubedSphereAdvectionSolver(config)

In [ ]:
# 2. Initialize State (Gaussian Hill)
def gaussian_hill(lon, lat):
    # Center at (0, 0)
    r = np.sqrt(lon**2 + lat**2)
    return np.exp( - (r / 0.5)**2 )

u0 = solver.get_initial_condition(type="custom", func=gaussian_hill)
print(f"Initial state shape: {u0.shape}")

In [ ]:
# 3. Run Simulation
t_final = 1.0
state = u0.copy()
t = 0.0

# Retrieve the auto-computed stable time step
# The solver computes dt based on the state and CFL
dt = solver.compute_safe_dt(state, cfl=target_cfl)
print(f"Computed Auto-dt: {dt:.6f}")

steps = int(t_final / dt)
print(f"Running for {steps} steps...")

for i in range(steps):
    state = solver.step(t, state, dt)
    t += dt 
    if (i+1) % 50 == 0:
        print(f"Step {i+1}/{steps} complete")

In [ ]:
# 4. Visualize Results
# We plot the field on Face 0 (Equatorial Face)
plt.figure(figsize=(8, 6))
plt.imshow(state[0, 0, :, :], origin='lower', extent=[-1, 1, -1, 1])
plt.colorbar(label='Tracer Concentration')
plt.title(f"Advection Result at t={t:.2f} (Face 0)")
plt.show()